In [2]:
import numpy as np
import matplotlib.pyplot as plt



In [3]:
order = np.load('orderings.npy')

In [12]:
N = order.shape[0]
iters = order.shape[1]

In [27]:
X = []
Y = []

for t in range(iters):
    
    states = np.zeros(N)
    
    
    for i in range(N):
        j = int(order[i,t])
        depart = np.zeros(N)
        depart[j] = 1
        X.append(states.copy())
        Y.append(depart)
        states[j]=1
        
    
    
X = np.array(X)
Y = np.array(Y)

In [29]:
X[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [31]:
Y[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0.])